In [33]:
import numpy as np
import numpy.random as npr  
import pandas as pd
import scipy.stats as si
from scipy import interpolate
from scipy.fftpack import fft, ifft
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'

In [34]:
S0 = 100.
K = 100.
r = 0.05
q = 0.02
sigma = 0.15

T = 10
M = 1000
I = 100000

N = 2048
x = 7.5

In [35]:
def d(S0, K, r, T, sigma, q):
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    #d2 = (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return (d1, d2)

def bsmprice(S0, K, r, T, sigma, q):
    ''' Valuation of European call option in BSM model.
    Analytical formula.

    Parameters
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        maturity date (in year fractions)
    r: float
        constant risk-free short rate
    sigma: float
        volatility factor in diffusion term

    Returns
    =======
    value: float
        present value of the European call option
    '''
    
    d1 = d(S0, K, r, T, sigma, q)[0]
    d2 = d(S0, K, r, T, sigma, q)[1]
    N_d1 = si.norm.cdf(d1, 0.0, 1.0)
    N_d2 = si.norm.cdf(d2, 0.0, 1.0)
    call = S0 * np.exp(-q * T) * N_d1 - K * np.exp(-r * T) * N_d2
    put = K * np.exp(-r * T) * (1 - N_d2) - S0 * np.exp(-q * T) * (1 - N_d1)
    return (call, put)

def bsmdelta(S0, K, r, T, sigma, q):
    d1 = d(S0, K, r, T, sigma, q)[0]
    N_d1 = si.norm.cdf(d1, 0.0, 1.0)
    delta_call = np.exp(-q * T) * N_d1
    delta_put = np.exp(-q * T) * (N_d1 - 1)
    return (delta_call, delta_put)

def bsmgamma(S0, K, r, T, sigma, q):
    d1 = d(S0, K, r, T, sigma, q)[0]
    phi_d1 = si.norm.pdf(d1, 0.0, 1.0)
    gamma_call = np.exp(-q * T) / (S0 * sigma * np.sqrt(T)) * phi_d1
    gamma_put = gamma_call
    return (gamma_call, gamma_put)

def bsmvega(S0, K, r, T, sigma, q):
    d1 = d(S0, K, r, T, sigma, q)[0]
    phi_d1 = si.norm.pdf(d1, 0.0, 1.0)
    vega_call = S0 *np.exp(-q * T) * np.sqrt(T) * phi_d1 #/ 100
    vega_put = vega_call
    return (vega_call, vega_put)

def bsmrho(S0, K, r, T, sigma, q):
    d2 = d(S0, K, r, T, sigma, q)[1]
    N_d2 = si.norm.cdf(d2, 0.0, 1.0)
    rho_call = K * T * np.exp(-r * T) * N_d2 #/ 100
    rho_put = -K * T * np.exp(-r * T) * (1 - N_d2) #/ 100
    return (rho_call, rho_put)

In [36]:
print(bsmprice(S0, K, r, T, sigma, q))
print(bsmdelta(S0, K, r, T, sigma, q))
print(bsmgamma(S0, K, r, T, sigma, q))
print(bsmrho(S0, K, r, T, sigma, q))
print(bsmvega(S0, K, r, T, sigma, q))

(26.48487206998933, 5.264862733454487)
(0.6613278706907427, -0.15740288238723912)
(0.004717830236169693, 0.004717830236169693)
(396.4791499908494, -210.05150972178404)
(70.76745354254543, 70.76745354254543)


In [37]:
def bsmprice_mc(S0, K, r, T, sigma, q, M, I):
    ''' Valuation of European options in BSM model
    by Monte Carlo simulation (of index level paths)
    
    Parameters
    ==========
    K: float
        (positive) strike price of the option
    option : string
        type of the option to be valued ('call', 'put')
    
    Returns
    =======
    C0: float
        estimated present value of European call option
    '''
    dt = T / M
    # simulation of index level paths
    S = np.zeros((M + 1, I))
    S[0] = S0
    sn = npr.standard_normal((M + 1, I))  
    for t in range(1, M + 1, 1):
        S[t] = S[t - 1] * np.exp((r - q - 0.5 * sigma ** 2) * dt 
                + sigma * np.sqrt(dt) * sn[t])     
    # case-based calculation of payoff
    cT = np.maximum(S[-1] - K, 0)
    pT = np.maximum(K - S[-1], 0)
    # calculation of MCS estimator
    call = np.exp(-r * T) * np.mean(cT)
    put = np.exp(-r * T) * np.mean(pT)
    return (call, put)

In [38]:
bsmprice_mc(S0, K, r, T, sigma, q, M, I)

(26.643118789215848, 5.243746532870301)

In [39]:
def bsmprice_fst(S0, K, r, T, sigma, q, N, x):
    x_max = x
    x_min = -x
    dx = (x_max - x_min) / (N-1)
    x = np.linspace(x_min, x_max, N)

    epsilon = 0.0001
    w_max = np.pi / dx
    dw = 2 * w_max / N
    w = np.hstack((np.arange(0, w_max+epsilon, dw), np.arange(-w_max+dw, -dw+epsilon, dw)))

    ST = S0 * np.exp(x)
    CT = np.maximum(ST - K, 0)
    PT = np.maximum(K - ST, 0)

    psi = 1j * (r-q-0.5*sigma**2) * w - 0.5 * (sigma*w)**2 - r
    char = np.exp(psi * T) # char_exp_factor
    C = ifft(fft(CT) * char).real
    P = ifft(fft(PT) * char).real
    
    cf = interpolate.PchipInterpolator(ST, C)
    call = cf(S0) + 0.0
    pf = interpolate.PchipInterpolator(ST, P)
    put = pf(S0) + 0.0
    return (call, put)

In [40]:
bsmprice_fst(S0, K, r, T, sigma, q, N, x)

(26.484977611428775, 5.264968302607745)

In [41]:
def bsmgreeks_fst(S0, K, r, T, sigma, q, N, x):
    x_max = x
    x_min = -x
    dx = (x_max - x_min) / (N-1)
    x = np.linspace(x_min, x_max, N)

    epsilon = 0.0001
    w_max = np.pi / dx
    dw = 2 * w_max / N
    w = np.hstack((np.arange(0, w_max+epsilon, dw), np.arange(-w_max+dw, -dw+epsilon, dw)))

    ST = S0 * np.exp(x)
    CT = np.maximum(ST - K, 0)
    PT = np.maximum(K - ST, 0)

    psi = 1j * (r-q-0.5*sigma**2) * w - 0.5 * (sigma*w)**2 - r
    char = np.exp(psi * T) # char_exp_factor

    de_c = (ifft(1j*w*fft(CT)*char)/ST).real
    ga_c = (ifft(-(1j*w+w**2)*fft(CT)*char)/ST**2).real
    rh_c = ifft((1j*w-1)*T*fft(CT)*char).real
    ve_c = ifft(-(1j*w+w**2)*sigma*T*fft(CT)*char).real

    de_p = (ifft(1j*w*fft(PT)*char)/ST).real
    ga_p = (ifft(-(1j*w+w**2)*fft(PT)*char)/ST**2).real
    rh_p = ifft((1j*w-1)*T*fft(PT)*char).real
    ve_p = ifft(-(1j*w+w**2)*sigma*T*fft(PT)*char).real
    
    f = interpolate.PchipInterpolator(ST, de_c)
    De_C = f(S0) + 0.0
    f = interpolate.PchipInterpolator(ST, ga_c)
    Ga_C = f(S0) + 0.0
    f = interpolate.PchipInterpolator(ST, rh_c)
    Rh_C = f(S0) + 0.0
    f = interpolate.PchipInterpolator(ST, ve_c)
    Ve_C = f(S0) + 0.0

    f = interpolate.PchipInterpolator(ST, de_p)
    De_P = f(S0) + 0.0
    f = interpolate.PchipInterpolator(ST, ga_p)
    Ga_P = f(S0) + 0.0
    f = interpolate.PchipInterpolator(ST, rh_p)
    Rh_P = f(S0) + 0.0
    f = interpolate.PchipInterpolator(ST, ve_p)
    Ve_P = f(S0) + 0.0

    return ((De_C, Ga_C, Rh_C, Ve_C), (De_P, Ga_P, Rh_P, Ve_P))

In [42]:
bsmgreeks_fst(S0, K, r, T, sigma, q, N, x)

((0.6613269906220355,
  0.0047177994679700975,
  396.47721454153697,
  70.76699283305207),
 (-0.15740376251782062,
  0.004717799467669321,
  -210.05344521308277,
  70.76699282853498))